In [33]:
import requests
import json
import os
from dotenv import load_dotenv
from urllib.parse import quote

# .env에서 API 키 불러오기
load_dotenv()
service_key = quote(os.getenv("SMARTFARM_API_KEY").strip(), safe='')

# 저장 경로
SAVE_PATH = "identity_data.json"

def get_identity_data_list():
    url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getIdentityDataList/{service_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # JSON 파일로 저장
        with open(SAVE_PATH, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"농가 정보가 '{SAVE_PATH}'에 저장되었습니다.")
        return data
    except Exception as e:
        print(f"API 요청 실패: {e}")
        return None

def get_cropping_season_data_list(service_key, user_id):
    encoded_key = quote(service_key, safe='')
    url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getCroppingSeasonDataList/{encoded_key}/{user_id}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"작기 정보 요청 실패: {e}")
        return None
    
def load_identity_data(filepath="identity_data.json"):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"파일 로딩 실패: {e}")
        return None
    
def get_cultivate_data_safe(args):
    user_id, serl_no, start, end = args
    url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getCultivateDataList/{service_key}/{user_id}/{serl_no}/{start}/{end}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        # ✅ statusCode가 "00"인 항목이 1개라도 있어야 유효
        if isinstance(data, list) and any(d.get("statusCode") == "00" for d in data):
            return {
                "userId": user_id,
                "croppingSerlNo": serl_no,
                "startDate": start,
                "endDate": end,
                "cultivateData": data
            }

    except Exception as e:
        print(f"❌ 실패: {user_id}, {serl_no} - {e}")
    return None

In [12]:
# 농가 목록을 조회하고 파일로 저장
if __name__ == "__main__":
    get_identity_data_list()

✅ 농가 정보가 'identity_data.json'에 저장되었습니다.


In [ ]:
## 토마토 작기 정보 operation 후 저장 (save file: tomato_cropping_data.json)

all_cropping_data = []

if __name__ == "__main__":
    identity_data = load_identity_data()

    if identity_data:
        tomato_farms = [entry for entry in identity_data if entry.get("itemCode") == "080300"] ## tomato key 080300
        print(f"토마토 농가 수: {len(tomato_farms)}")

        for farm in tomato_farms:
            user_id = farm.get("userId")
            print(f"\n userId: {user_id} - 작기 정보 조회 중...")
            cropping_data = get_cropping_season_data_list(service_key, user_id)

            if cropping_data:
                all_cropping_data.append({
                    "userId": user_id,
                    "croppingData": cropping_data
                })
    else:
        print("농가 정보를 불러오지 못했습니다.")

with open("tomato_cropping_data.json", "w", encoding="utf-8") as f:
    json.dump(all_cropping_data, f, ensure_ascii=False, indent=2)

🍅 토마토 농가 수: 355

📦 userId: PFS_0000001 - 작기 정보 조회 중...

📦 userId: PFS_0000002 - 작기 정보 조회 중...

📦 userId: PFS_0000003 - 작기 정보 조회 중...

📦 userId: PFS_0000004 - 작기 정보 조회 중...

📦 userId: PFS_0000006 - 작기 정보 조회 중...

📦 userId: PFS_0000007 - 작기 정보 조회 중...

📦 userId: PFS_0000008 - 작기 정보 조회 중...

📦 userId: PFS_0000009 - 작기 정보 조회 중...

📦 userId: PF_0000002 - 작기 정보 조회 중...

📦 userId: PF_0000004 - 작기 정보 조회 중...

📦 userId: PF_0000005 - 작기 정보 조회 중...

📦 userId: PF_0000006 - 작기 정보 조회 중...

📦 userId: PF_0000007 - 작기 정보 조회 중...

📦 userId: PF_0000008 - 작기 정보 조회 중...

📦 userId: PF_0000009 - 작기 정보 조회 중...

📦 userId: PF_0000010 - 작기 정보 조회 중...

📦 userId: PF_0000011 - 작기 정보 조회 중...

📦 userId: PF_0000012 - 작기 정보 조회 중...

📦 userId: PF_0000013 - 작기 정보 조회 중...

📦 userId: PF_0000014 - 작기 정보 조회 중...

📦 userId: PF_0000015 - 작기 정보 조회 중...

📦 userId: PF_0000016 - 작기 정보 조회 중...

📦 userId: PF_0000017 - 작기 정보 조회 중...

📦 userId: PF_0000018 - 작기 정보 조회 중...

📦 userId: PF_0000019 - 작기 정보 조회 중...

📦 userId: PF_0000020 - 작기

In [30]:
#check data
user_id = "PFS_0000001"             # 예시: 실제 userId
cropping_serl_no = 79               # 작기 일련번호
start_date = "2015-09-02"          # 작기 시작일
end_date = "2016-12-31"            # 작기 종료일

# user_id = "PFS_0000008"             # 예시: 실제 userId # 데이터 없는 경우  "statusMessage": "NODATA_ERROR",
# cropping_serl_no = 77               # 작기 일련번호
# start_date = "2015-07-29"          # 작기 시작일
# end_date = "2016-12-31"            # 작기 종료일

# URL 구성
url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getCultivateDataList/{service_key}/{user_id}/{cropping_serl_no}/{start_date}/{end_date}"

# 요청 및 출력
try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    data = response.json()
    if data:
        print("✅ 생육 데이터 존재:")
        print(json.dumps(data, indent=2, ensure_ascii=False))
    else:
        print("🚫 생육 데이터 없음")
except Exception as e:
    print(f"❌ 요청 실패: {e}")

✅ 생육 데이터 존재:
[
  {
    "statusCode": "00",
    "statusMessage": "NORMAL_CODE",
    "userId": "PFS_0000001",
    "itemCode": "080300",
    "sampleNum": "1",
    "measDate": "2015-09-06",
    "growLength": 210.0,
    "flowerTop": 210.0,
    "stemDiameter": 15.0,
    "leavesLength": 450.0,
    "leavesWidth": 430.0,
    "leavesNum": 17.0,
    "flowerPosition": 0.0,
    "fruitsPosition": 0.0,
    "fruitsNum": 0.0,
    "harvestPosition": 0.0,
    "ped": 5451.0,
    "solarCorrection": 1716.0,
    "fruitsWeight": 180.0
  },
  {
    "statusCode": null,
    "statusMessage": null,
    "userId": "PFS_0000001",
    "itemCode": "080300",
    "sampleNum": "1",
    "measDate": "2015-09-13",
    "growLength": 210.0,
    "flowerTop": 210.0,
    "stemDiameter": 14.0,
    "leavesLength": 455.0,
    "leavesWidth": 430.0,
    "leavesNum": 17.0,
    "flowerPosition": 3.0,
    "fruitsPosition": 3.0,
    "fruitsNum": 3.0,
    "harvestPosition": 0.0,
    "ped": 4092.0,
    "solarCorrection": 3624.0,
    "fruits

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

with open("tomato_cropping_data.json", "r", encoding="utf-8") as f:
    tomato_cropping_data = json.load(f)

# 요청 목록 생성
request_args = []
for entry in tomato_cropping_data:
    user_id = entry.get("userId")
    for crop in entry.get("croppingData", []):
        serl_no = crop.get("croppingSerlNo")
        start = crop.get("croppingDate")
        end = crop.get("croppingEndDate")
        if all([user_id, serl_no, start, end]):
            request_args.append((user_id, serl_no, start, end))

# 병렬 실행
valid_entries = []
total = len(request_args)
completed = 0

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(get_cultivate_data_safe, args) for args in request_args]
    for future in as_completed(futures):
        completed += 1
        result = future.result()
        if result:
            valid_entries.append(result)
            print(f"✅ {completed}/{total} | 생육 데이터 있음: userId={result['userId']} serlNo={result['croppingSerlNo']}")
        else:
            print(f"❌ {completed}/{total} | 생육 데이터 없음")

# JSON 저장
with open("./data/tomato_cultivate_data_valid.json", "w", encoding="utf-8") as f:
    json.dump(valid_entries, f, ensure_ascii=False, indent=2)

# 요약 출력
print(f"\n🎯 최종 요약")
print(f"✅ 생육 데이터 있는 항목: {len(valid_entries)}개")
print(f"🚫 생육 데이터 없는 항목: {total - len(valid_entries)}개")

❌ 1/553 | 생육 데이터 없음
✅ 2/553 | 생육 데이터 있음: userId=PFS_0000008 serlNo=71
✅ 3/553 | 생육 데이터 있음: userId=PFS_0000008 serlNo=536
✅ 4/553 | 생육 데이터 있음: userId=PFS_0000002 serlNo=80
✅ 5/553 | 생육 데이터 있음: userId=PFS_0000001 serlNo=79
✅ 6/553 | 생육 데이터 있음: userId=PFS_0000001 serlNo=4940
✅ 7/553 | 생육 데이터 있음: userId=PFS_0000006 serlNo=69
✅ 8/553 | 생육 데이터 있음: userId=PFS_0000004 serlNo=82
✅ 9/553 | 생육 데이터 있음: userId=PFS_0000007 serlNo=70
✅ 10/553 | 생육 데이터 있음: userId=PFS_0000003 serlNo=81
❌ 11/553 | 생육 데이터 없음
❌ 12/553 | 생육 데이터 없음
❌ 13/553 | 생육 데이터 없음
✅ 14/553 | 생육 데이터 있음: userId=PFS_0000008 serlNo=4809
✅ 15/553 | 생육 데이터 있음: userId=PFS_0000008 serlNo=5377
❌ 16/553 | 생육 데이터 없음
✅ 17/553 | 생육 데이터 있음: userId=PF_0000002 serlNo=204
✅ 18/553 | 생육 데이터 있음: userId=PFS_0000009 serlNo=72
✅ 19/553 | 생육 데이터 있음: userId=PF_0000002 serlNo=43
✅ 20/553 | 생육 데이터 있음: userId=PF_0000004 serlNo=38
✅ 21/553 | 생육 데이터 있음: userId=PF_0000005 serlNo=87
✅ 22/553 | 생육 데이터 있음: userId=PF_0000004 serlNo=88
✅ 23/553 | 생육 데이터 있음: userId=PF_00

In [ ]:
# 생육 + 환경데이터 있는 세트 불러오기
# EI = 내부환경데이터
EI_CODES = [
    "TI", "HI", "HI01", "HI02", "ID", "IR", "IS", "LI", "LW", "CI", "DT", "WS"
]

def get_env_data(service_key, facility_id, meas_date, item_code, fatr_code,
                 fld_code="FG", sect_code="EI"):
    url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getEnvDataList/" \
          f"{service_key}/{facility_id}/{meas_date}/{fld_code}/{sect_code}/{fatr_code}/{item_code}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return data if data else None
    except Exception as e:
        print(f"❌ 환경 정보 실패: {facility_id}, {meas_date}, {fatr_code} → {e}")
        return None
    
# 1. load 생육 데이터와 농가정보 매핑
with open("tomato_cultivate_data_valid.json", "r", encoding="utf-8") as f:
    grow_data = json.load(f)

with open("identity_data.json", "r", encoding="utf-8") as f:
    identity_data = json.load(f)

# 2. userId → facilityId 매핑
user_facility = {d["userId"]: d["facilityId"] for d in identity_data if d.get("userId") and d.get("facilityId")}

# 3. 반복 수집
env_entries = []
for entry in grow_data:
    user_id = entry["userId"]
    facility_id = user_facility.get(user_id)
    if not facility_id:
        continue

    for grow_point in entry["cultivateData"]:
        meas_date = grow_point.get("measDate")  # yyyy-mm-dd
        item_code = grow_point.get("itemCode", "080300")
        if meas_date:
            for fatr_code in EI_CODES:
                env = get_env_data(service_key, facility_id, meas_date, item_code, fatr_code)
                if env:
                    env_entries.append({
                        "userId": user_id,
                        "facilityId": facility_id,
                        "measDate": meas_date,
                        "fatrCode": fatr_code,
                        "envData": env
                    })

# 4. 저장
with open("./data/tomato_env_data.json", "w", encoding="utf-8") as f:
    json.dump(env_entries, f, ensure_ascii=False, indent=2)



❌ 환경 정보 실패: PFS_0000002_01, 2016-02-14, IR → 502 Server Error: Bad Gateway for url: https://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getEnvDataList/94cec237e4b943bfbb0e499d779025df/PFS_0000002_01/2016-02-14/FG/EI/IR/080300
❌ 환경 정보 실패: PFS_0000008_01, 2021-12-15, HI → 502 Server Error: Bad Gateway for url: https://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getEnvDataList/94cec237e4b943bfbb0e499d779025df/PFS_0000008_01/2021-12-15/FG/EI/HI/090100
❌ 환경 정보 실패: PF_0002528_01, 2022-06-29, IS → HTTPSConnectionPool(host='www.smartfarmkorea.net', port=443): Read timed out. (read timeout=10)
❌ 환경 정보 실패: PF_0002528_01, 2018-10-29, DT → HTTPSConnectionPool(host='www.smartfarmkorea.net', port=443): Read timed out. (read timeout=10)
❌ 환경 정보 실패: PF_0002528_01, 2019-04-22, TI → 502 Server Error: Bad Gateway for url: https://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getEnvDataList/94cec237e4b943bfbb0e499d779025df/PF_0002528_01/2019-04-22/FG/